In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#Load the trained model, scaler, pickle, onehotencoder, labelencoder
model= load_model('model.h5')

#Load the laber encoder for gender column
with open('label_encoder.pkl','rb') as file:
    label_encoder=pickle.load(file)

#load the one_hot_encoder for geography column
with open('one_hot_encoder.pkl','rb') as file:
    one_hot_encoder=pickle.load(file)

#Load the scaler pressoing for all column
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)





In [3]:
#Example of input data
input_data ={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':5000
    
}

In [4]:
#One hot encoding the geography column
geo_encoded = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(['Geography']))

g:\generative ai project\ann_project\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
#Converting the above key value pair into dataframe
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,5000


In [7]:
## label encode the gender column
input_df['Gender']=label_encoder.transform(input_df['Gender'])
input_df.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,5000


In [9]:
#now concat the geographical df with input_df
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df.head(1)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,5000,1.0,0.0,0.0


In [11]:
#Now scaling the scalar data
input_scaled= scaler.transform(input_df)
input_scaled

array([[-0.52042638,  0.9104886 ,  0.09806986, -0.69042789, -0.27370895,
         0.79729637,  0.6474524 ,  0.96607511, -1.65165649,  1.00025003,
        -0.58312392, -0.57176909]])

In [12]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step


array([[0.03516418]], dtype=float32)

In [13]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.035164177)

In [14]:
if prediction_proba>0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
